In [20]:
#installed the langchain library

!pip install langchain


In [21]:
#installed the langchain community version

!pip install langchain_community


In [22]:
!pip install --upgrade langchain langchain_community  #just upgraded the library

In [23]:
!pip install langchain-groq  #installed langchain groq

In [ ]:
from langchain_groq import ChatGroq

llm=ChatGroq(
    temperature=0,
    groq_api_key="use your key",
    model_name="llama-3.1-70b-versatile"

)


In [25]:
response = llm.invoke("the first person to land on the moon was...")
print(response)

content='Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission. His famous words upon landing were, "That\'s one small step for man, one giant leap for mankind."' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 45, 'total_tokens': 93, 'completion_time': 0.192, 'prompt_time': 0.007787328, 'queue_time': 0.005619901, 'total_time': 0.199787328}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None} id='run-692fcfe9-5c95-4b6a-940d-3ff9166718c6-0' usage_metadata={'input_tokens': 45, 'output_tokens': 48, 'total_tokens': 93}


In [26]:
pip install chromadb


In [29]:
import chromadb

client=chromadb.Client()
collection =client.create_collection(name="my_collection1")


In [30]:
collection.add(
    documents=[
        "this document is about football",
        "argentina is the world cup winner"
    ],
    ids=['id1','id2']

)

In [31]:
all_documents=collection.get()
print(all_documents)

{'ids': ['id1', 'id2'], 'embeddings': None, 'documents': ['this document is about football', 'argentina is the world cup winner'], 'uris': None, 'data': None, 'metadatas': [None, None], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [32]:
results=collection.query(
    query_texts=["who is messi"],
    n_results=2


)

results

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['argentina is the world cup winner',
   'this document is about football']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.3602893352508545, 1.681389331817627]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [36]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.accenture.com/in-en/careers/jobdetails?id=ATCI-4611634-S1808710_en&title=Application%20Developer")
page_data = loader.load().pop().page_content
print(page_data)








Application Developer












































































Skip to main content
Skip to footer




























Insights


Services


















Cloud
			  
		  


Cybersecurity
			  
		  


Data and Artificial Intelligence
			  
		  


Digital Engineering and Manufacturing
			  
		  


Emerging Technology
			  
		  


Enterprise Platforms
			  
		  


Finance and Risk Management
			  
		  


Learning
			  
		  


Marketing and Experience
			  
		  


Metaverse
			  
		  


Private Equity
			  
		  


Sales and Commerce
			  
		  


Strategic Managed Services
			  
		  


Strategy
			  
		  


Supply Chain
			  
		  


Sustainability
			  
		  


Talent and Organization
			  
		  


Technology Transformation
			  
		  




















Industries


















Aerospace and Defense
			  
		  


Automotive
			  
		  


 Banking
			  
		  


 Capital Markets
			  
		  


Chemicals
			  
		  


 Communications and Media
		

In [64]:
from langchain_core.prompts import PromptTemplate

promt_extract=PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION:
    the scraped text from the careers's page of a website.
    your job is to extract the job postings and return them in JSON format containing the following
    keys: `role`,'expirience`,`skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NOPREAMBLE):
    """

)

chain_extract=promt_extract | llm
res = chain_extract.invoke({"page_data":page_data})
print(res.content)


[
  {
    "role": "Application Developer",
    "experience": "Minimum 3 year(s) of experience is required",
    "skills": {
      "must_have": ["Spring Boot"],
      "good_to_have": ["Java", "RESTful APIs", "microservices architecture"]
    },
    "description": "Design, build, and configure applications to meet business process and application requirements. Develop and implement software solutions using Spring Boot. Collaborate with cross-functional teams, analyze user requirements, design and code applications, and ensure the quality and performance of the software."
  }
]


In [43]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)
type(json_res)


[{'role': 'Application Developer', 'experience': 'Minimum 3 year(s) of experience is required', 'skills': {'must_have': ['Spring Boot'], 'good_to_have': ['Java', 'RESTful APIs', 'microservices architecture']}, 'description': 'Design, build, and configure applications to meet business process and application requirements. Develop and implement software solutions using Spring Boot. Collaborate with cross-functional teams, analyze user requirements, design and code applications, and ensure the quality and performance of the software.'}]


list

In [45]:
type(json_res)

list

In [48]:
import pandas as pd

df=pd.read_csv('my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [51]:

from uuid import uuid4
import chromadb
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name='portfolio')

if not collection.count():
  for _,row in df.iterrows():
    collection.add(documents=row["Techstack"],
                   metadatas={"lLnks" : row["Links"]},
                   ids=[str(uuid4())])

In [56]:
links=collection.query(query_texts=["Experience in Python","Expertise in Java "],n_results=2).get('metadatas')
links


[[{'lLnks': 'https://example.com/ml-python-portfolio'},
  {'lLnks': 'https://example.com/python-portfolio'}],
 [{'lLnks': 'https://example.com/java-portfolio'},
  {'lLnks': 'https://example.com/android-portfolio'}]]

In [60]:
job= json_res
for item in job:
    print(item['skills'])

{'must_have': ['Spring Boot'], 'good_to_have': ['Java', 'RESTful APIs', 'microservices architecture']}


In [65]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Application Development Services for Your Business Needs

Dear [Client's Name],

I hope this email finds you well. As a Business Development Executive at AtliQ, I came across your job posting for an Application Developer and was impressed by the role's requirements. I'd like to introduce you to AtliQ's capabilities in fulfilling your needs.

At AtliQ, we specialize in AI & Software Consulting, empowering enterprises with tailored solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in designing, building, and configuring applications to meet business process and application requirements.

Specifically, our team has hands-on experience with Spring Boot, Java, RESTful APIs, and microservices architecture, which aligns perfectly with your job requirements. We can help you develop and implement software solutions that meet your business needs.

I'd like to showcase some of our